# Preprocessing 

## Importing Necessary Libraries

In [1]:
#!pip install plotly

In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import re
import plotly.figure_factory as ff
import plotly.express as px

## Loading Data

In [3]:
df = pd.read_csv('games_info.csv')
game_details = pd.read_csv('game_details.csv')

## Initial Data Exploration

In [4]:
df.head()

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,SUPPORTED_PLATFORMS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,RELEASE_DATE,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK
0,Dragon's Dogma: Dark Arisen,"Store Genres Action (1), Adventure (25), RPG (3)",review_score 8,7 Languages,Capcom,Windows,39271,5015,NaN,15 January 2016 – 16:48:00 UTC (9 years ago),"27,368\nall-time peak 9 years ago17 January 2016",$22.49,3.0,"1,064\n24-hour peak"
1,Forza Horizon 5,"Store Genres Action (1), Adventure (25), Racin...",review_score 8,24 Languages,Playground Games,Windows,156683,21075,NaN,9 November 2021 – 04:58:00 UTC (3 years ago),"81,096\nall-time peak 3 years ago13 November 2021",$32.78,53.0,"16,131\n24-hour peak"
2,Thrive,"Store Genres Casual (4), Indie (23), Simulatio...",review_score 8,1 Languages,Revolutionary Games Studio,Windows Linux,872,70,Engine.Godot,26 November 2021 – 10:54:48 UTC (3 years ago),145\nall-time peak 2 years ago24 July 2022,$2.99,NaN,21\n24-hour peak
3,Layers of Fear (2016),"Store Genres Adventure (25), Indie (23)",review_score 8,12 Languages,Bloober Team SA,Windows macOS Linux,22766,2819,Engine.Unity,15 February 2016 – 21:53:00 UTC (8 years ago),"145,384\nall-time peak 6 years ago13 June 2018",$10.19,3.0,31\n24-hour peak
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",review_score 8,1 Languages,lazarche,Windows,834,66,"Detected Technologies (?), SDK.SteamworksNET",25 May 2023 – 10:22:41 UTC (12 months ago),"3,778\nall-time peak 10 months ago18 July 2023",NaN,4.0,318\n24-hour peak


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259 entries, 0 to 2258
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2259 non-null   object 
 1   STORE_GENRE            2221 non-null   object 
 2   RATING_SCORE           2225 non-null   object 
 3   N_SUPPORTED_LANGUAGES  2226 non-null   object 
 4   DEVELOPERS             2258 non-null   object 
 5   SUPPORTED_PLATFORMS    2197 non-null   object 
 6   POSITIVE_REVIEWS       2259 non-null   int64  
 7   NEGATIVE_REVIEWS       2259 non-null   int64  
 8   TECHNOLOGIES           1845 non-null   object 
 9   RELEASE_DATE           2255 non-null   object 
 10  TOTAL_TWITCH_PEAK      2259 non-null   object 
 11  PRICE                  1507 non-null   object 
 12  N_DLC                  789 non-null    float64
 13  24_HOUR_PEAK           2208 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 247.2+

In [6]:
df.describe()

,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,N_DLC
count,2.259000e+03,2259.000000,789.000000
mean,2.087847e+04,1270.529438,5.904943
std,7.540080e+04,5824.763668,17.345104
min,3.400000e+01,0.000000,1.000000
25%,7.245000e+02,33.000000,1.000000
50%,2.509000e+03,136.000000,2.000000
75%,1.043100e+04,602.000000,4.000000
max,1.257272e+06,190953.000000,379.000000


In [7]:
df.describe(include='object')

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,SUPPORTED_PLATFORMS,TECHNOLOGIES,RELEASE_DATE,TOTAL_TWITCH_PEAK,PRICE,24_HOUR_PEAK
count,2259,2221,2225,2226,2258,2197,1845,2255,2259,1507,2208
unique,2088,325,3,36,1668,6,234,2071,2066,125,643
top,TOEM,"Store Genres Action (1), Adventure (25), Indie...",review_score 8,1 Languages,Square Enix,Windows,Engine.Unity,17 September 2021 – 13:00:40 UTC (3 years ago),2 years ago,$19.99,4\n24-hour peak
freq,4,113,1355,583,16,1294,764,4,8,206,81


## Removing Duplicated Games

In [8]:
df_copy = df.copy()
duplicate_rows = df[df.duplicated(subset=['NAME'])]
print("Duplicate Rows based on the 'NAME' column:")
duplicate_rows
num_duplicates = duplicate_rows.shape[0]
print(f"Number of duplicate rows based on the 'NAME' column: {num_duplicates}")
df = df_copy.drop_duplicates(subset=['NAME'], keep='first')

# df_copy = df.copy()
# for col in df_copy.columns:
#     if df_copy[col].apply(lambda x: isinstance(x, list)).any():
#         df_copy[col] = df_copy[col].apply(tuple)
# duplicate_rows = df_copy[df_copy.duplicated()]
# num_duplicates = duplicate_rows.shape[0]
# print(f"Number of duplicate rows: {num_duplicates}")
# df = df_copy.drop_duplicates(keep='first')
# duplicate_rows

Duplicate Rows based on the 'NAME' column:
Number of duplicate rows based on the 'NAME' column: 171


## Handling Missing Values

In [9]:
df.replace('N/A', pd.NA, inplace=True)

## Extracting and Handling 'PUBLISH_YEAR' from 'RELEASE_DATE'

This section extracts the year from the 'RELEASE_DATE' column to create a new 'PUBLISH_YEAR' column. It handles any non-numeric values and missing data by converting them to a consistent format, and finally replaces null values in 'PUBLISH_YEAR' with the median value.

In [10]:
if 'RELEASE_DATE' in df.columns:
    df['PUBLISH_YEAR'] = df['RELEASE_DATE'].str.extract(r'(\d{4})')

    df['PUBLISH_YEAR'] = pd.to_numeric(df['PUBLISH_YEAR'], errors='coerce')

    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].fillna(-1)
    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].astype(int)
    df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].replace(-1, pd.NA)

    df = df.drop('RELEASE_DATE', axis=1)

### Replace null values in 'PUBLISH_YEAR' with the median of the column

In [11]:
median_publish_year = df['PUBLISH_YEAR'].median()
filled_publish_year = df['PUBLISH_YEAR'].fillna(median_publish_year)

C:\Users\AFRAA\AppData\Local\Temp\ipykernel_16256\2151296842.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filled_publish_year = df['PUBLISH_YEAR'].fillna(median_publish_year)


In [12]:
null_count_before = df['PUBLISH_YEAR'].isnull().sum()

In [13]:
print("Null count in PUBLISH_YEAR column before dropping:", null_count_before)

Null count in PUBLISH_YEAR column before dropping: 4


In [14]:
df.dropna(subset=['PUBLISH_YEAR'], inplace=True)

In [15]:
null_count_after = df['PUBLISH_YEAR'].isnull().sum()
print("Null count in PUBLISH_YEAR column after dropping:", null_count_after)

Null count in PUBLISH_YEAR column after dropping: 0


In [16]:
df['PUBLISH_YEAR'] = df['PUBLISH_YEAR'].astype(int)

## Removing Rows with Missing 'DEVELOPERS' Data

In [17]:
df.dropna(subset=['DEVELOPERS'], inplace=True)

## Cleaning and Converting 'N_SUPPORTED_LANGUAGES' Data

This section handles the `N_SUPPORTED_LANGUAGES` column by filling missing values with a default number (-1) and converting the column to an integer type after removing any extraneous text.

In [18]:
df['N_SUPPORTED_LANGUAGES'] = df['N_SUPPORTED_LANGUAGES'].fillna(1)

In [19]:
df['N_SUPPORTED_LANGUAGES'] = df['N_SUPPORTED_LANGUAGES'].astype(str).str.replace(' Languages', '').astype(int)

In [20]:
placeholder_value = 'review_score -1'
df['RATING_SCORE'] = df['RATING_SCORE'].fillna(placeholder_value)

In [21]:
df['RATING_SCORE'] = df['RATING_SCORE'].str.replace('review_score ', '').astype(float).astype(int)  

In [22]:
mean_rating_score = df.loc[df['RATING_SCORE'] != -1, 'RATING_SCORE'].mean()  
df.loc[df['RATING_SCORE'] == -1, 'RATING_SCORE'] = mean_rating_score

C:\Users\AFRAA\AppData\Local\Temp\ipykernel_16256\2311779403.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.391516333495856' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[df['RATING_SCORE'] == -1, 'RATING_SCORE'] = mean_rating_score


In [23]:
df.head()

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,SUPPORTED_PLATFORMS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR
0,Dragon's Dogma: Dark Arisen,"Store Genres Action (1), Adventure (25), RPG (3)",8.0,7,Capcom,Windows,39271,5015,NaN,"27,368\nall-time peak 9 years ago17 January 2016",$22.49,3.0,"1,064\n24-hour peak",2016
1,Forza Horizon 5,"Store Genres Action (1), Adventure (25), Racin...",8.0,24,Playground Games,Windows,156683,21075,NaN,"81,096\nall-time peak 3 years ago13 November 2021",$32.78,53.0,"16,131\n24-hour peak",2021
2,Thrive,"Store Genres Casual (4), Indie (23), Simulatio...",8.0,1,Revolutionary Games Studio,Windows Linux,872,70,Engine.Godot,145\nall-time peak 2 years ago24 July 2022,$2.99,NaN,21\n24-hour peak,2021
3,Layers of Fear (2016),"Store Genres Adventure (25), Indie (23)",8.0,12,Bloober Team SA,Windows macOS Linux,22766,2819,Engine.Unity,"145,384\nall-time peak 6 years ago13 June 2018",$10.19,3.0,31\n24-hour peak,2016
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",8.0,1,lazarche,Windows,834,66,"Detected Technologies (?), SDK.SteamworksNET","3,778\nall-time peak 10 months ago18 July 2023",NaN,4.0,318\n24-hour peak,2023


## One-Hot Encoding 'SUPPORTED_PLATFORMS' Column

This section transforms the 'SUPPORTED_PLATFORMS' column into separate binary columns for each platform using one-hot encoding, which facilitates easier analysis and modeling.

In [24]:
# Split the 'SUPPORTED_PLATFORMS' column into separate columns for each platform
platform_dummies = df['SUPPORTED_PLATFORMS'].str.get_dummies(sep=' ')

# Concatenate the dummy columns with the original DataFrame
df = pd.concat([df, platform_dummies], axis=1)

# Drop the original 'SUPPORTED_PLATFORMS' column
df.drop(columns=['SUPPORTED_PLATFORMS'], inplace=True)

In [25]:
df.head()

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR,Linux,Windows,macOS
0,Dragon's Dogma: Dark Arisen,"Store Genres Action (1), Adventure (25), RPG (3)",8.0,7,Capcom,39271,5015,NaN,"27,368\nall-time peak 9 years ago17 January 2016",$22.49,3.0,"1,064\n24-hour peak",2016,0,1,0
1,Forza Horizon 5,"Store Genres Action (1), Adventure (25), Racin...",8.0,24,Playground Games,156683,21075,NaN,"81,096\nall-time peak 3 years ago13 November 2021",$32.78,53.0,"16,131\n24-hour peak",2021,0,1,0
2,Thrive,"Store Genres Casual (4), Indie (23), Simulatio...",8.0,1,Revolutionary Games Studio,872,70,Engine.Godot,145\nall-time peak 2 years ago24 July 2022,$2.99,NaN,21\n24-hour peak,2021,1,1,0
3,Layers of Fear (2016),"Store Genres Adventure (25), Indie (23)",8.0,12,Bloober Team SA,22766,2819,Engine.Unity,"145,384\nall-time peak 6 years ago13 June 2018",$10.19,3.0,31\n24-hour peak,2016,1,1,1
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",8.0,1,lazarche,834,66,"Detected Technologies (?), SDK.SteamworksNET","3,778\nall-time peak 10 months ago18 July 2023",NaN,4.0,318\n24-hour peak,2023,0,1,0


## Handling 'PRICE' Column with Missing Values and Formatting

This section addresses the missing values and formatting issues in the 'PRICE' column. It involves filling missing prices from a secondary dataset, replacing placeholders and formatting symbols, and converting the column to a numeric type.

In [26]:
game_details['price'] = game_details['price'].replace('-', '$0')

In [27]:
# Iterate over rows with null PRICE in the main DataFrame
for index, row in df[df['PRICE'].isnull()].iterrows():
    game_name = row['NAME']
    # Find corresponding price in game_details DataFrame
    price = game_details[game_details['name'] == game_name]['price'].values

    # If price exists in game_details DataFrame, fill in the PRICE column of the main DataFrame
    if (len(price) > 0):
        #print(price[0])
        df.at[index, 'PRICE'] = price[0]

In [28]:
df[df['PRICE'].isnull() == True]

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR,Linux,Windows,macOS
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",8.0,1,lazarche,834,66,"Detected Technologies (?), SDK.SteamworksNET","3,778\nall-time peak 10 months ago18 July 2023",NaN,4.0,318\n24-hour peak,2023,0,1,0
5,U-ena -遠花火の少女-,"Store Genres Adventure (25), Casual (4), Free ...",8.0,3,HemiolaStudio,582,41,"Engine.Lightvn, Detected Technologies (?), Eng...",2 years ago,NaN,NaN,NaN,2022,0,1,0
9,Rocket League,"Store Genres Action (1), Indie (23), Racing (9...",8.0,13,Psyonix LLC,508263,70214,"Engine.Unreal, SDK.EpicOnlineServices","147,632\nall-time peak 4 years ago27 September...",NaN,25.0,"32,851\n24-hour peak",2015,1,1,1
10,Wurroom,"Store Genres Adventure (25), Casual (4), Free ...",8.0,1,Michael Rfdshir,603,43,"Engine.Unity, Detected Technologies (?), Engin...",5 years ago,NaN,1.0,NaN,2019,0,1,0
11,Kitten adventures in city park,"Store Genres Adventure (25), Indie (23)",8.0,6,Lokator Studio,541,37,"Engine.PyGame, Engine.RenPy, Detected Technolo...",30\nall-time peak 3 years ago27 November 2021,NaN,1.0,1\n24-hour peak,2017,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,WTF Do You Know?,"Store Genres Indie (23), RPG (3), Simulation (28)",8.0,1,Randumb Studios,122,6,Engine.RPGMaker,10\nall-time peak 2 years ago2 May 2022,NaN,NaN,1\n24-hour peak,2022,0,1,0
2238,From Madness with Love,"Store Genres Adventure (25), Casual (4), Indie...",8.0,4,Jamsanpoid,223,17,Engine.TyranoBuilder,132\nall-time peak 11 months ago18 July 2023,NaN,NaN,14\n24-hour peak,2023,0,1,1
2240,The Forest Quartet,"Store Genres Adventure (25), Indie (23)",8.0,14,Mads & Friends,205,15,Engine.Unity,10\nall-time peak 2 years ago9 December 2022,NaN,NaN,3\n24-hour peak,2022,0,1,0
2241,Purple Place - Classic Games,"Store Genres Casual (4), Indie (23), Simulatio...",8.0,26,Sam Cohle,1223,150,Engine.Unity,58\nall-time peak 7 months ago22 October 2023,NaN,NaN,17\n24-hour peak,2023,1,1,1


In [29]:
null_count = df['PRICE'].isnull().sum()

print("Null count in PRICE column:", null_count)

Null count in PRICE column: 251


In [30]:
# Drop rows with null values in the 'PRICE' column
#df.dropna(subset=['PRICE'], inplace=True)
df['PRICE'] = df['PRICE'].fillna('$0')

In [31]:
df[df['PRICE'].isnull() == True]

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR,Linux,Windows,macOS


In [32]:
null_count = df['PRICE'].isnull().sum()

print("Null count in PRICE column:", null_count)

Null count in PRICE column: 0


In [33]:
#df['PRICE'] = df['PRICE'].str.replace('€', '.')

#### Extract the price from the "price" column using regex


In [34]:
price_pattern = r'\$([\d.]+)'

df['PRICE'] = df['PRICE'].str.extract(price_pattern)

In [35]:
df.head()

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR,Linux,Windows,macOS
0,Dragon's Dogma: Dark Arisen,"Store Genres Action (1), Adventure (25), RPG (3)",8.0,7,Capcom,39271,5015,NaN,"27,368\nall-time peak 9 years ago17 January 2016",22.49,3.0,"1,064\n24-hour peak",2016,0,1,0
1,Forza Horizon 5,"Store Genres Action (1), Adventure (25), Racin...",8.0,24,Playground Games,156683,21075,NaN,"81,096\nall-time peak 3 years ago13 November 2021",32.78,53.0,"16,131\n24-hour peak",2021,0,1,0
2,Thrive,"Store Genres Casual (4), Indie (23), Simulatio...",8.0,1,Revolutionary Games Studio,872,70,Engine.Godot,145\nall-time peak 2 years ago24 July 2022,2.99,NaN,21\n24-hour peak,2021,1,1,0
3,Layers of Fear (2016),"Store Genres Adventure (25), Indie (23)",8.0,12,Bloober Team SA,22766,2819,Engine.Unity,"145,384\nall-time peak 6 years ago13 June 2018",10.19,3.0,31\n24-hour peak,2016,1,1,1
4,TaskPals,"Store Genres Indie (23), Free to Play (37)",8.0,1,lazarche,834,66,"Detected Technologies (?), SDK.SteamworksNET","3,778\nall-time peak 10 months ago18 July 2023",0,4.0,318\n24-hour peak,2023,0,1,0


In [36]:
df[df['PRICE'].isnull() == True]

,NAME,STORE_GENRE,RATING_SCORE,N_SUPPORTED_LANGUAGES,DEVELOPERS,POSITIVE_REVIEWS,NEGATIVE_REVIEWS,TECHNOLOGIES,TOTAL_TWITCH_PEAK,PRICE,N_DLC,24_HOUR_PEAK,PUBLISH_YEAR,Linux,Windows,macOS
511,Don't Pee,"Store Genres Adventure (25), Simulation (28)",8.000000,1,Naughty Clogs,163,11,Engine.Unreal,23\nall-time peak 4 months ago11 February 2024,NaN,NaN,2\n24-hour peak,2024,0,1,0
512,Ratopia,"Store Genres Adventure (25), Casual (4), Indie...",8.000000,8,Cassel Games,2043,275,Engine.Unity,"5,772\nall-time peak 7 months ago14 November 2023",NaN,NaN,256\n24-hour peak,2023,0,1,1
513,Command & Conquer™ Generals Zero Hour,"Store Genres Action (1), Strategy (2)",8.000000,9,EA Los Angeles,1384,177,NaN,"2,353\nall-time peak 2 months ago17 March 2024",NaN,NaN,"1,200\n24-hour peak",2024,0,1,0
516,Team Fortress Classic,Store Genres Action (1),8.000000,9,Valve,7079,1070,Engine.GoldSource,"1,897\nall-time peak 20 years ago9 August 2004",NaN,NaN,95\n24-hour peak,1999,1,1,1
517,Make Way,"Store Genres Action (1), Casual (4), Indie (23...",8.000000,10,Ice BEAM,418,42,Engine.Unity,347\nall-time peak last month28 April 2024,NaN,NaN,155\n24-hour peak,2023,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,ISEKAI QUEST,NaN,8.391516,1,Studio Ginkgo,3235,438,Engine.Unity,4 years ago,NaN,2.0,NaN,2020,0,1,1
625,Street Legal Racing: Redline v2.3.1,NaN,8.391516,1,Invictus Games Ltd.,3304,448,NaN,7 years ago,NaN,1.0,NaN,2016,0,1,0
627,Dead Grid,"Store Genres Indie (23), RPG (3), Strategy (2)...",8.000000,1,ATOM VOID,264,21,Engine.Unity,194\nall-time peak 2 years ago26 March 2022,NaN,NaN,4\n24-hour peak,2022,0,1,0
629,Nexomon,"Store Genres Adventure (25), Casual (4), Indie...",8.000000,1,VEWO Interactive Inc.,1314,159,Engine.Unity,208\nall-time peak 4 years ago12 July 2020,NaN,NaN,17\n24-hour peak,2020,0,1,1


In [37]:
df.dropna(subset=['PRICE'], inplace=True)

In [38]:
df['PRICE'] = df['PRICE'].astype(float)

## Cleaning 'STORE_GENRE' Column

This section focuses on cleaning the 'STORE_GENRE' column. It fills missing values with a placeholder, removes unnecessary text, and splits the genre strings for better usability.

In [39]:
def clean_store_genre(genre_string):
    genres = genre_string.split(', ')
    cleaned_genres = []
    for genre in genres:
        cleaned_genre = genre.split(' (')[0]
        cleaned_genres.append(cleaned_genre)
    return cleaned_genres

df['STORE_GENRE'] = df['STORE_GENRE'].fillna('Store Genres')
df['STORE_GENRE'] = df['STORE_GENRE'].str.replace('Store Genres', '')
df['STORE_GENRE'] = df['STORE_GENRE'].apply(clean_store_genre)


## Cleaning '24_HOUR_PEAK' Column


This section handles the cleaning and conversion of the '24_HOUR_PEAK' column. It fills missing values, splits the strings to extract numerical values, removes commas, and converts the column to an integer type.

In [40]:
df['24_HOUR_PEAK']

0        1,064\n24-hour peak
1       16,131\n24-hour peak
2           21\n24-hour peak
3           31\n24-hour peak
4          318\n24-hour peak
                ...         
2253        17\n24-hour peak
2254        13\n24-hour peak
2256       254\n24-hour peak
2257        61\n24-hour peak
2258       106\n24-hour peak
Name: 24_HOUR_PEAK, Length: 2004, dtype: object

In [41]:
df['24_HOUR_PEAK'] = df['24_HOUR_PEAK'].fillna(("0\n24-hour peak"))
df['24_HOUR_PEAK'] = df['24_HOUR_PEAK'].apply(lambda x: int(x.split('\n')[0].replace(',', ''))).astype(int)

In [42]:
df['24_HOUR_PEAK']

0        1064
1       16131
2          21
3          31
4         318
        ...  
2253       17
2254       13
2256      254
2257       61
2258      106
Name: 24_HOUR_PEAK, Length: 2004, dtype: int32

## Cleaning 'TECHNOLOGIES' Column

This section addresses the 'TECHNOLOGIES' column by filling missing values with an empty string and splitting the comma-separated technologies into lists for easier analysis.

In [43]:
df['TECHNOLOGIES']

0                                                NaN
1                                                NaN
2                                       Engine.Godot
3                                       Engine.Unity
4       Detected Technologies (?), SDK.SteamworksNET
                            ...                     
2253                                    Engine.Unity
2254                                             NaN
2256                                  Engine.Solar2D
2257             AntiCheat.PunkBuster, Engine.idTech
2258                                    Engine.Unity
Name: TECHNOLOGIES, Length: 2004, dtype: object

In [44]:
df['TECHNOLOGIES'] = df['TECHNOLOGIES'].fillna('')  # Replace NaN with empty string
df['TECHNOLOGIES'] = df['TECHNOLOGIES'].apply(lambda x: x.split(', ') if x else [])

In [45]:
df['TECHNOLOGIES']

0                                                   []
1                                                   []
2                                       [Engine.Godot]
3                                       [Engine.Unity]
4       [Detected Technologies (?), SDK.SteamworksNET]
                             ...                      
2253                                    [Engine.Unity]
2254                                                []
2256                                  [Engine.Solar2D]
2257             [AntiCheat.PunkBuster, Engine.idTech]
2258                                    [Engine.Unity]
Name: TECHNOLOGIES, Length: 2004, dtype: object

## Cleaning 'TOTAL_TWITCH_PEAK' Column

This section cleans the 'TOTAL_TWITCH_PEAK' column by splitting it into two new columns: 'TWITCH_PEAK_HOUR' and 'TWITCH_PEAK_YEAR'. It handles missing values and converts the data to appropriate numeric types.

In [46]:
df['TOTAL_TWITCH_PEAK']

0        27,368\nall-time peak 9 years ago17 January 2016
1       81,096\nall-time peak 3 years ago13 November 2021
2              145\nall-time peak 2 years ago24 July 2022
3          145,384\nall-time peak 6 years ago13 June 2018
4          3,778\nall-time peak 10 months ago18 July 2023
                              ...                        
2253    1,434\nall-time peak 8 years ago18 September 2016
2254         118\nall-time peak 4 years ago25 August 2020
2256         4,026\nall-time peak 2 years ago3 April 2022
2257             96\nall-time peak 4 years ago22 May 2020
2258    1,115\nall-time peak 3 months ago25 February 2024
Name: TOTAL_TWITCH_PEAK, Length: 2004, dtype: object

In [47]:
if 'TOTAL_TWITCH_PEAK' in df.columns:
    # Extract and convert the TWITCH_PEAK_HOUR
    df['TWITCH_PEAK_HOUR'] = df['TOTAL_TWITCH_PEAK'].apply(lambda x: x.split('\n')[0].replace(',', ''))
    df['TWITCH_PEAK_HOUR'] = pd.to_numeric(df['TWITCH_PEAK_HOUR'], errors='coerce')
    df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].fillna(-1)
    df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].astype(int)
   # df['TWITCH_PEAK_HOUR'] = df['TWITCH_PEAK_HOUR'].replace(-1, pd.NA)

    # Extract and convert the TWITCH_PEAK_YEAR
    df['TWITCH_PEAK_YEAR'] = df['TOTAL_TWITCH_PEAK'].apply(lambda x: x.split()[-1])
    df['TWITCH_PEAK_YEAR'] = pd.to_numeric(df['TWITCH_PEAK_YEAR'], errors='coerce')
    df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].fillna(-1)
    df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].astype(int)
    #df['TWITCH_PEAK_YEAR'] = df['TWITCH_PEAK_YEAR'].replace(-1, pd.NA)

    # Drop the original column
    df.drop(columns=['TOTAL_TWITCH_PEAK'], inplace=True)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2004 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2004 non-null   object 
 1   STORE_GENRE            2004 non-null   object 
 2   RATING_SCORE           2004 non-null   float64
 3   N_SUPPORTED_LANGUAGES  2004 non-null   int32  
 4   DEVELOPERS             2004 non-null   object 
 5   POSITIVE_REVIEWS       2004 non-null   int64  
 6   NEGATIVE_REVIEWS       2004 non-null   int64  
 7   TECHNOLOGIES           2004 non-null   object 
 8   PRICE                  2004 non-null   float64
 9   N_DLC                  703 non-null    float64
 10  24_HOUR_PEAK           2004 non-null   int32  
 11  PUBLISH_YEAR           2004 non-null   int32  
 12  Linux                  2004 non-null   int64  
 13  Windows                2004 non-null   int64  
 14  macOS                  2004 non-null   int64  
 15  TWITCH_PE

## Cleaning 'N_DLC' Column

This section handles the cleaning of the 'N_DLC' column by converting it to a numeric type, handling missing values, and calculating the percentage of null values in the column. It also drops the column if necessary.



In [49]:
df['N_DLC'] = pd.to_numeric(df['N_DLC'], errors='coerce')  # Ensure numeric conversion
df['N_DLC'] = df['N_DLC'].fillna(-1)  # Temporarily fill NaNs with -1
df['N_DLC'] = df['N_DLC'].astype(int)  # Convert to int
df['N_DLC'] = df['N_DLC'].replace(-1, pd.NA)  # Restore NaNs
df['N_DLC']

0          3
1         53
2       <NA>
3          3
4          4
        ... 
2253    <NA>
2254    <NA>
2256    <NA>
2257    <NA>
2258       1
Name: N_DLC, Length: 2004, dtype: object

In [50]:
column_name = 'N_DLC'
num_nulls = df[column_name].isnull().sum()
total_entries = len(df[column_name])
null_percentage = (num_nulls / total_entries) * 100
print(f"Percentage of null values in column '{column_name}': {null_percentage:.2f}%")


Percentage of null values in column 'N_DLC': 64.92%


In [51]:
df.drop(columns=['N_DLC'], inplace=True)

## Calculating 'TOTAL_REVIEW' Column

This section calculates a new 'TOTAL_REVIEW' column, which represents the proportion of positive reviews out of the total number of reviews (positive and negative). This helps in understanding the overall review distribution for each game.

In [52]:
df['TOTAL_REVIEW'] = df['POSITIVE_REVIEWS'] / (df['NEGATIVE_REVIEWS'] + df['POSITIVE_REVIEWS'])

## Final Dataset Preparation

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2004 entries, 0 to 2258
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NAME                   2004 non-null   object 
 1   STORE_GENRE            2004 non-null   object 
 2   RATING_SCORE           2004 non-null   float64
 3   N_SUPPORTED_LANGUAGES  2004 non-null   int32  
 4   DEVELOPERS             2004 non-null   object 
 5   POSITIVE_REVIEWS       2004 non-null   int64  
 6   NEGATIVE_REVIEWS       2004 non-null   int64  
 7   TECHNOLOGIES           2004 non-null   object 
 8   PRICE                  2004 non-null   float64
 9   24_HOUR_PEAK           2004 non-null   int32  
 10  PUBLISH_YEAR           2004 non-null   int32  
 11  Linux                  2004 non-null   int64  
 12  Windows                2004 non-null   int64  
 13  macOS                  2004 non-null   int64  
 14  TWITCH_PEAK_HOUR       2004 non-null   int32  
 15  TWITCH_PE

In [54]:
df.to_csv('preprocessed_game_info.csv', index=False)